# Active Base in Dark

In [18]:
#importing table 'BIGTABLE(Details)'and read as pandas.....
import pandas as pd
Details = pd.read_csv('Bigtable.csv',usecols = ['Full SFID','Contract Id','Type','System ID','Customer Signed By: Full Name','Customer Signed By: Phone','Customer: Secondary Phone number','Customer Warranty End Date','Price Book: Price Book Name','Contract Start Date','Status','StatusII','Customer Status'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
#importing table 'DARK REPORT'(Dark) and read as pandas.....
#Reading the required excel sheets and columns from the BI Daily Reports. The selected columns are considered valid for the required task
r1 = pd.read_excel('BI NG daily reports of 2023-02-07.xlsx', 'Contracts', usecols = ['Contract','SaleCH','generation','LTOperiod','LTOunits','type'])

r2 = pd.read_excel('BI NG daily reports of 2023-02-07.xlsx','Sps Contracts', usecols = ['Contract','activation_start','activation_end'])

r3 = pd.read_excel('BI NG daily reports of 2023-02-07.xlsx','Contracts per PaidPeriod', usecols = ['Contract','PaidPeriods'])

r4 = pd.read_excel('BI NG daily reports of 2023-02-07.xlsx','CRM contracts', names = ['Contract','CRMcontract'])

r5 = pd.read_excel('BI NG daily reports of 2023-02-07.xlsx','Last payments', usecols = ['Contract','payment_date','activation_end'])

r5.columns = ['Contract','last_payment_date','lastpayment_activation_end']

In [20]:
r2['activation_start'] = pd.to_datetime(r2['activation_start']).dt.normalize()
r2['activation_end'] = pd.to_datetime(r2['activation_end']).dt.normalize()
r5['lastpayment_activation_end'] = pd.to_datetime(r5['lastpayment_activation_end'],errors = 'coerce')
r5['last_payment_date'] = pd.to_datetime(r5['last_payment_date']).dt.normalize()
r5['lastpayment_activation_end'] = pd.to_datetime(r5['lastpayment_activation_end']).dt.normalize()

In [21]:
check1 = pd.merge(pd.merge(pd.merge(pd.merge(r1, r2, how='left', on='Contract'), r3, how='left', on='Contract'), r4, how='left', on='Contract'), r5, how='left', on='Contract')
check1 = pd.merge(check1, Details, how='left', left_on='CRMcontract', right_on='Contract Id')

check1 = check1[check1['Type'] == 'Production']

check1["ref_activation_end"] = check1[["activation_end","lastpayment_activation_end"]].max(axis=1)

check1['ref_activation_end'] = check1['ref_activation_end'].fillna('1900-01-01')

check1['ref_activation_end'] = pd.to_datetime(check1['ref_activation_end']).dt.normalize()

In [22]:
import pandas as pd
import datetime as date
from datetime import date
from datetime import timedelta

today = pd.to_datetime('today').normalize() - timedelta(days=1)

check1['Outage_days'] = (check1['ref_activation_end'] - today).dt.days

In [23]:
check1['Customer Warranty End Date'] = pd.to_datetime(check1['Customer Warranty End Date'], format='%d/%m/%Y')

check1['Contract Start Date'] = pd.to_datetime(check1['Contract Start Date'], format='%d/%m/%Y')

In [24]:
import numpy as np
filters = [
   (check1['Price Book: Price Book Name'].str.contains("NWO") == True)
]
values = ['LCP']

check1["lcp_status"] = np.select(filters, values, default='Non-LCP')

In [36]:
import numpy as np

filters = [
    (check1.Outage_days > -1),
   (check1.Outage_days > -60) & (check1.Outage_days < 0)
]
values = ['Paid','Active']
check1["Old Status"] = np.select(filters, values, default= 'Churn')

In [37]:
check2 = check1[['Contract','CRMcontract','lcp_status','Outage_days','Type','Old Status','System ID','SaleCH','generation','Customer Signed By: Full Name','Customer Signed By: Phone',
               'Customer: Secondary Phone number','type','Contract Start Date','Price Book: Price Book Name','Status','StatusII','Customer Status']]

In [42]:
check3 = check2[((check2['Old Status'] == 'Active')  & ((check2['Price Book: Price Book Name'] != 'Direct-NG (NWO)') & (check2['Status'] != 'Cancelled') & (check2['StatusII'] != 'Freeing') & (check2['Customer Status'] != 'Owner') & (check2['Status'] != 'Canceled') & (check2['StatusII'] != 'Free') & (check2['Type'] == 'Production')))]



In [43]:
check3.drop('Type', axis = 1, inplace=True)
check3.drop('Price Book: Price Book Name', axis = 1, inplace=True)
check3.drop('Status', axis = 1, inplace=True)
check3.drop('StatusII', axis = 1, inplace=True)
check3.drop('Customer Status', axis = 1, inplace=True)

In [44]:
check3.to_excel('Active Base in Dark.xlsx')